<h1>Trabalho Prático 1 - Daniel Pimentel Kansaon</h1>


O trabalho consiste na aplicação de algoritmos supervisionados comparando ao todo 6 métodos:

- **Naive Bayes**: Apenas um experimento para servir de baseline.
- **Decision Tree**: Variar a altura máxima da árvore (incluindo permitir altura ilimitada) e mostrar os resultados graficamente.
- **SVM**: Avaliar os kernels linear, sigmoid, polinomial e RBF.
- **k-NN**: Variar o número k de vizinhos e mostrar os resultados graficamente.
- **Random Forest**: Variar o número de árvores e mostrar os resultados graficamente.
- **Gradient Tree Boosting**: Variar o número de itera¸c˜oes e mostrar os resultados graficamente.

Os métodos serão testados em um problema de classificação binária (0 - 1) de candidatos a exoplanetas encontrados pela sonda espacial Kepler da NASA. Um exoplaneta é um planeta fora do sistema solar (i.e. que não orbita o sol). A sonda primeiro identifica sinais de possíıveis exoplanetas, chamados de Kepler Object of Interest (KOI). Porem nem todos os KOIs são de fato exoplanetas, alguns se tratam de "Falsos Positivos" de origens diversas. A tarefa é então classificar os KOIs entre exoplanetas
confirmados e "Falsos Positivos". Cada observação corresponde a um KOI e as features são características estimadas de cada (possível) exoplaneta (tamanho, temperatura, features da estrela hospedeira, etc).


In [108]:
#Adicionando as referencias que serão usadas durante o trabalho

import pandas as pd
import numpy as np
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import feature_selection
import statistics

#Metodos
from sklearn import svm
from sklearn import neighbors #K-nn
from sklearn import ensemble #Random Forest
from sklearn.naive_bayes import GaussianNB #Naive Bayes
from sklearn.naive_bayes import MultinomialNB #Naive Bayes


In [2]:
# As constantes do projeto estão adicionadas aqui.

# Caminho arquivo com os dados
FILE_PATH = "koi_data.csv"

#Tamanho do fold que será utilizado para a etapa de Cross-Validation
K_FOLD = 5

#Coluna TARGET do arquivo CSV
TARGET = "koi_disposition"

#Coluna que possui o nome do KOI
COLUMN_KOI = "kepoi_name"

## Tratamento nos Dados

Como dito acima, cada observação realizada corresponde a um KOI e as features são características estimadas de cada (possível) exoplaneta (tamanho, temperatura, features da estrela hospedeira, etc).

Assim, nesta etapa serão realizados processamentos nos dados a fim de prepará-los para a execução dos algoritmos.

In [119]:
df = pd.read_csv(FILE_PATH)

# Imprime algumas informacoes basicas sobre os dados
print("Numero de linhas: {}".format(df.shape[0]))
print("Numero de colunas: {}".format(df.shape[1]))

# Cria a lista de features com base nas colunas do arquivo CSV
features = list(df.columns)

#Remove a coluna target pois ela que está querendo prever.
features.remove(TARGET)

#Remove a coluna de nome, pois não é necessária para a predição
features.remove(COLUMN_KOI)

Numero de linhas: 5202
Numero de colunas: 43


## Ajustando o Target (rótulos dos dados)

In [4]:
#Porcentagem de False Positive e Confirmed 
display((df[TARGET].value_counts()/len(df)*100).round(2).to_frame(TARGET +" (%)").T)

df[TARGET] = (df[TARGET] == "CONFIRMED").astype(int)

,FALSE POSITIVE,CONFIRMED
koi_disposition (%),59.55,40.45


## Separação dos dados para a validação cruzada

Nessa parte são separados os dados com base nos folds(5) para a validação cruzada. A partir disto, os algoritmos irão usar essa separação para os testes.

In [61]:
#Criando os vetores de treino e teste
x_train = []
y_train = []
x_validation = []
y_validation = []

#Selecionando as features
#df = df.sample(20000, replace=False, random_state=1)
skb = feature_selection.SelectKBest(feature_selection.f_classif, 41)
skb = skb.fit(df[features], df[TARGET])

#separa o X são os exemplos e o Y é o rótulo [0 ou 1]
X = df
Y = df[TARGET]

#Aqui é realizado é feito a separação dos dados em K_FOLD = 5.
data_kfold = StratifiedKFold(n_splits=K_FOLD)
data_kfold.get_n_splits(X, Y)


#Nessa parte é lido os folds e armazenado nas listas.
# Exemplo: x_train [ fold1, fold2, fold3, fold4, fold5 ]

for train_index, test_index in data_kfold.split(X, Y):
    x_train.append(skb.transform(df.loc[train_index, features]))
    x_validation.append(skb.transform(df.loc[test_index, features]))
    y_train.append(df.loc[train_index, TARGET])
    y_validation.append(df.loc[test_index, TARGET])    

## Naive Bayes

In [118]:
def executar_naivebayes():
    train_acc = []
    val_acc = []
    
    gnb = GaussianNB()
    
    for i in range(0, K_FOLD):            
        gnb = gnb.fit(x_train[i], y_train[i]) #Treino
        train_acc.append(gnb.score(x_train[i], y_train[i]))
        val_acc.append(gnb.score(x_validation[i], y_validation[i]))   
          
    print("Média da Acurácia na treino: {:.3}".format(np.average(train_acc)))
    print("Média da Acurácia na validação: {:.3}".format(np.average(val_acc)))
    print("Desvio Padrão da Acurácia na treino: {:.3}".format(statistics.stdev(train_acc)))
    print("Desvio Padrão da Acurácia na validação: {:.3}".format(statistics.stdev(val_acc))) 
    
    # https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [107]:
executar_naivebayes()

GaussianNB(priors=None, var_smoothing=1e-09)
Média da Acurácia na treino: 0.8
Média da Acurácia na validação: 0.803
Desvio Padrão da Acurácia na treino: 0.0118
Desvio Padrão da Acurácia na validação: 0.0514


## k-NN

In [6]:
def executar_k_nn(k): 
    train_acc = []
    val_acc = []
    
    print("K = {}".format(k))             
    clf = neighbors.KNeighborsClassifier(n_neighbors=k) 
    
    for i in range(0, K_FOLD):          
        clf = clf.fit(x_train[i], y_train[i])  
        train_acc.append(clf.score(x_train[i], y_train[i]))
        val_acc.append(clf.score(x_validation[i], y_validation[i]))

    print("Média da Acurácia na treino: {:.3}".format(np.average(train_acc)))
    print("Média da Acurácia na validação: {:.3}".format(np.average(val_acc)))
    print("Desvio Padrão da Acurácia na treino: {:.3}".format(statistics.stdev(train_acc)))
    print("Desvio Padrão da Acurácia na validação: {:.3}".format(statistics.stdev(val_acc)))   
            

In [7]:
executar_k_nn(5)

K = 5
Média da Acurácia na treino: 0.848
Média da Acurácia na validação: 0.773
Desvio Padrão da Acurácia na treino: 0.00781
Desvio Padrão da Acurácia na validação: 0.0482


In [8]:
executar_k_nn(10)

K = 10
Média da Acurácia na treino: 0.825
Média da Acurácia na validação: 0.776
Desvio Padrão da Acurácia na treino: 0.0109
Desvio Padrão da Acurácia na validação: 0.0456


In [31]:
executar_k_nn(32)

K = 32
Média da Acurácia na treino: 0.799
Média da Acurácia na validação: 0.781
Desvio Padrão da Acurácia na treino: 0.011
Desvio Padrão da Acurácia na validação: 0.0539


## Support Vector Machine

In [32]:
def executar_svm_linar(c):    
    clf = svm.SVC(c, "linear")
    
    for i in range(0, 1):          
        clf = clf.fit(x_train[i], y_train[i])  
        train_acc.append(clf.score(x_train[i], y_train[i]))
        val_acc.append(clf.score(x_validation[i], y_validation[i]))
   
    print("Média da Acurácia na treino: {:.3}".format(np.average(train_acc)))
    print("Média da Acurácia na validação: {:.3}".format(np.average(val_acc)))
    print("Desvio Padrão da Acurácia na treino: {:.3}".format(statistics.stdev(train_acc)))
    print("Desvio Padrão da Acurácia na validação: {:.3}".format(statistics.stdev(val_acc)))   

In [ ]:
executar_svm_linar(0.01)

## Random Forest

In [40]:
def executar_ramdomforest(N, max_depth):
    train_acc = []
    val_acc = []
    
    clf = ensemble.RandomForestClassifier(n_estimators=N, max_depth=max_depth)
    
    for i in range(0, K_FOLD):          
        clf = clf.fit(x_train[i], y_train[i])  
        train_acc.append(clf.score(x_train[i], y_train[i]))
        val_acc.append(clf.score(x_validation[i], y_validation[i]))        

    print("Média da Acurácia na treino: {:.3}".format(np.average(train_acc)))
    print("Média da Acurácia na validação: {:.3}".format(np.average(val_acc)))
    print("Desvio Padrão da Acurácia na treino: {:.3}".format(statistics.stdev(train_acc)))
    print("Desvio Padrão da Acurácia na validação: {:.3}".format(statistics.stdev(val_acc)))

In [46]:
executar_ramdomforest(50, None)

Média da Acurácia na treino: 1.0
Média da Acurácia na validação: 0.966
Desvio Padrão da Acurácia na treino: 0.000107
Desvio Padrão da Acurácia na validação: 0.0133


In [60]:
executar_ramdomforest(25, 10)

Média da Acurácia na treino: 0.991
Média da Acurácia na validação: 0.965
Desvio Padrão da Acurácia na treino: 0.00192
Desvio Padrão da Acurácia na validação: 0.0131
